#Information
Group 9 : Resume parser

Content : Classification of resumes using Bert

In [1]:
# Connect to drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Move the directory to the csv files
%cd /content/drive/MyDrive/NLP_Final_changes

/content/drive/MyDrive/NLP_Final_changes


In [3]:
# packages
import pandas as pd
import numpy as np
import seaborn as sns
import re
import math
import matplotlib.pyplot as plt

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import  word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [1]:
import pandas as pd

In [3]:
ls

Bert_Similarity.ipynb
Bert_classification.ipynb
Category_plot.png
Preprocessing.ipynb
ResumeDataSet.csv
TF_IDF_Classification.ipynb
TF_IDF_Similarity.ipynb
Word2Vec.ipynb
Word_cloud.png
cleaned_resume_final.csv
csv/
resume_classification_model/
sample_data_science_job_description.docx
sample_data_science_resumes.csv
sample_java_job_description.docx
sample_java_resumes.csv
sample_testing_job_description.docx
sample_testing_resumes.csv


In [5]:
ls

Bert_Similarity.ipynb
Bert_classification.ipynb
Category_plot.png
Preprocessing.ipynb
ResumeDataSet.csv
TF_IDF_Classification.ipynb
TF_IDF_Similarity.ipynb
Word2Vec.ipynb
Word_cloud.png
cleaned_resume_final.csv
csv/
resume_classification_model/
sample_data_science_job_description.docx
sample_data_science_resumes.csv
sample_java_job_description.docx
sample_java_resumes.csv
sample_testing_job_description.docx
sample_testing_resumes.csv


In [6]:
# Load required CSV files

df=pd.read_csv("./csv/cleaned_resume_final.csv")# please changed this to correct relatice pathc 
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [6]:
df['Category_no']=df.groupby('Category').ngroup()

In [7]:
df.head()

,Category,Resume,cleaned_resume,Category_no
0,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...,6
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education detail may may data scientist data s...,6
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",area interest deep learning control system des...,6
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skill r python sap hana tableau sap hana sql s...,6
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education detail mca ymcaust faridabad haryana...,6


In [8]:
# Tensorflow imports
import tensorflow as tf
import tensorflow_hub as hub

In [9]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.8 MB/s eta 0:00:00


In [10]:
from tqdm.auto import tqdm
from transformers import BertTokenizer

In [11]:
# Import bert based tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [37]:
# function to generate input and mask ids based on input data
# A fixed vector size of 256 is used, you can modify it based on your requirement
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['cleaned_resume'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [38]:
# Vector size should be same for input and mask ids
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [39]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [40]:
categories = df['Category'].unique().shape[0]
labels = np.zeros((len(df), categories))
labels.shape

(962, 25)

In [41]:
# Creating a sparse matrix with 1 for each category
labels[np.arange(len(df)), df['Category_no'].values] = 1

In [42]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))

In [43]:
def resume_classification(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [44]:
# Mapping input with the respective category labels
dataset = dataset.map(resume_classification)

In [45]:
# shuffle and batch can be varied based on the size of the input data
dataset = dataset.shuffle(3).batch(2, drop_remainder=True)

In [46]:
# Train and test split with 80 and 20 percecnt
p = 0.8
train_size = int((len(df)//categories)*p)
train_size

30

In [47]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [48]:
# import bert model
from transformers import TFBertModel

In [49]:
# bert base model with pretrained weights
model = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [50]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(df['Category_no'].unique().shape[0], activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

resume_classification_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
resume_classification_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

In [51]:
resume_classification_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [52]:
# number of epochs can be varied based on the data set
hist = resume_classification_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

Epoch 1/10
30/30 [==============================] - 73s 1s/step - loss: 1.1521 - accuracy: 0.9000 - val_loss: 22.7816 - val_accuracy: 0.0266
Epoch 2/10
30/30 [==============================] - 23s 769ms/step - loss: 6.5427 - accuracy: 0.6667 - val_loss: 20.3033 - val_accuracy: 0.0266
Epoch 3/10
30/30 [==============================] - 26s 881ms/step - loss: 1.8409 - accuracy: 0.6000 - val_loss: 10.2723 - val_accuracy: 0.0266
Epoch 4/10
30/30 [==============================] - 21s 729ms/step - loss: 3.3972 - accuracy: 0.7333 - val_loss: 30.4724 - val_accuracy: 0.0266
Epoch 5/10
30/30 [==============================] - 20s 698ms/step - loss: 6.2459 - accuracy: 0.3667 - val_loss: 4.9344 - val_accuracy: 0.0000e+00
Epoch 6/10
30/30 [==============================] - 21s 723ms/step - loss: 0.9865 - accuracy: 0.7500 - val_loss: 9.8886 - val_accuracy: 0.0266
Epoch 7/10
30/30 [==============================] - 20s 697ms/step - loss: 1.6324 - accuracy: 0.3667 - val_loss: 9.0202 - val_accuracy: 0

In [28]:
# save the trained model to save time and resources
resume_classification_model.save('resume_classification_model')

In [34]:
# load trained model and use it to predict the given data
resume_classification_model = tf.keras.models.load_model('resume_classification_model')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=df['Category'].unique().tolist()):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [35]:
# test data
input_text = df['cleaned_resume'][0]
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(resume_classification_model, processed_data=processed_data)
print(f"Predicted Category: {result}")

1/1 [==============================] - 3s 3s/step
Predicted Category: Sales
